# Proof of concept DWWC matrix computation

In [1]:
import pandas
from neo4j.v1 import GraphDatabase
import hetio.readwrite
import hetio.neo4j
import hetio.pathtools

from hetmech.degree_weight import dwwc
from hetmech.matrix import get_node_to_position

In [2]:
url = 'https://github.com/dhimmel/hetionet/raw/76550e6c93fbe92124edc71725e8c7dd4ca8b1f5/hetnet/json/hetionet-v1.0.json.bz2'
graph = hetio.readwrite.read_graph(url)
metagraph = graph.metagraph

In [12]:
compound = 'DB01156'  # Bupropion
disease = 'DOID:0050742'  # nicotine dependences

damping_exponent = 0.4

# CbGpPWpGaD contains duplicate metanodes, so DWPC is not equivalent to DWPC
metapath = metagraph.metapath_from_abbrev('CbGpPWpGaD')
metapath.get_unicode_str()

compound_to_position = {x.identifier: i for x, i in get_node_to_position(graph, 'Compound').items()}
disease_to_position = {x.identifier: i for x, i in get_node_to_position(graph, 'Disease').items()}
i = compound_to_position[compound]
j = disease_to_position[disease]

### Cypher DWPC implementation

In [13]:
%%time
query = hetio.neo4j.construct_dwpc_query(metapath, property='identifier', unique_nodes=True)
print(query)

driver = GraphDatabase.driver("bolt://neo4j.het.io")
params = {
    'source': compound,
    'target': disease,
    'w': damping_exponent,
}
with driver.session() as session:
    result = session.run(query, params)
    result = result.single()
result

MATCH path = (n0:Compound)-[:BINDS_CbG]-(n1)-[:PARTICIPATES_GpPW]-(n2)-[:PARTICIPATES_GpPW]-(n3)-[:ASSOCIATES_DaG]-(n4:Disease)
USING JOIN ON n2
WHERE n0.identifier = { source }
AND n4.identifier = { target }
AND n1 <> n3
WITH
[
size((n0)-[:BINDS_CbG]-()),
size(()-[:BINDS_CbG]-(n1)),
size((n1)-[:PARTICIPATES_GpPW]-()),
size(()-[:PARTICIPATES_GpPW]-(n2)),
size((n2)-[:PARTICIPATES_GpPW]-()),
size(()-[:PARTICIPATES_GpPW]-(n3)),
size((n3)-[:ASSOCIATES_DaG]-()),
size(()-[:ASSOCIATES_DaG]-(n4))
] AS degrees, path
RETURN
count(path) AS PC,
sum(reduce(pdp = 1.0, d in degrees| pdp * d ^ -{ w })) AS DWPC
CPU times: user 22.8 ms, sys: 13 ms, total: 35.8 ms
Wall time: 204 ms


In [14]:
cypher_pc = result['PC']
print(cypher_pc)
cypher_dwpc = result['DWPC']
print(cypher_dwpc)

142
0.03287590886921623


### hetio DWPC implementation

In [15]:
%%time
compound_id = 'Compound', compound
disease_id = 'Disease', disease
hetio_paths = hetio.pathtools.paths_between(
    graph, 
    source=graph.node_dict[compound_id],
    target=graph.node_dict[disease_id],
    metapath=metapath,
    duplicates=False,
)

# Path count
print(len(hetio_paths))

# DWPC
hetio_dwpc = hetio.pathtools.DWPC(hetio_paths, damping_exponent=damping_exponent)

142
CPU times: user 244 ms, sys: 1.99 ms, total: 246 ms
Wall time: 246 ms


In [16]:
hetio_dwpc

0.03287590886921622

### HetMech dwpc

In [37]:
# from hetmech.path_count import dwpc

import numpy
import collections
from scipy.sparse import csr_matrix
import scipy.sparse

from hetmech.matrix import metaedge_to_adjacency_matrix
from hetmech.degree_weight import dwwc_step


def dwpc(graph, metapath, damping=1.0, verbose=False):
    """
    Compute the degree-weighted path count (DWPC).
    First checks that no more than one metanode type is repeated.
    """
    print("Calling DWPC")
    dwpc_matrix = None
    # First read through sequence of metanode types
    nodetype_sequence = collections.defaultdict(int)
    for metaedge in metapath:
        nodetype_sequence[metaedge.source] += 1
        nodetype_sequence[metaedge.target] += 1
    # Identify the repeated node type
    # and check that no more than 1 metanodetype is repeated
    repeated_node = list(filter(lambda x: nodetype_sequence[x] > 2,
                           nodetype_sequence))
    if len(repeated_node) > 1:
        if verbose:
            print("Input metapath repeats more than one nodetype")
        return dwpc_matrix

    elif len(repeated_node) == 0:
        if verbose:
            print("Input metapath has no repeated nodetypes")
        # Now perform multiplications
        for metaedge in metapath:
            adj_mat = metaedge_to_adjacency_matrix(graph, metaedge)
            adj_mat = dwwc_step(adj_mat, damping, damping)
            if dwpc_matrix is None:
                dwpc_matrix = csr_matrix(adj_mat).copy()
            else:
                dwpc_matrix = dwpc_matrix @ adj_mat
    else:
        # Determine start/endpoints for left,loop,right
        if verbose:
            print("Input metapath repeats exactly one nodetype")
            print("Repeated node list: {}, type {}".format(repeated_node, repeated_node[0]))
        
        first_appearance = None
        last_appearance = None
        for idx, metaedge in enumerate(metapath):
            if repeated_node[0] == metaedge.source:
                if first_appearance is None:
                    first_appearance = idx
            if repeated_node[0] == metaedge.target:
                last_appearance = idx
        if verbose:
            print("metapath has {} nodes, and {} are the repeated ones"
                  "".format(len(metapath),[first_appearance, last_appearance]))

        # Handle head
        left_matrix = None
        for idx, metaedge in enumerate(metapath):
            if verbose:
                print("\tWorking on {}".format(idx))
            adj_mat = metaedge_to_adjacency_matrix(graph, metaedge)
            adj_mat = dwwc_step(adj_mat, damping, damping)
            adj_mat = csr_matrix(adj_mat)
            # metanodes = metaedge.get_nodes()
            if idx < first_appearance:  # haven't seen repeatednode yet
                if verbose:
                    print("\t\tleft_matrix {}".format(idx))
                if left_matrix is None:
                    if verbose:
                        print("\t\tleft_matrix {} initialize; shape adj {}".format(idx, adj_mat.shape))
                    left_matrix = adj_mat.copy()
                else:
                    if verbose:
                        print("\t\tleft_matrix {} multiply; shape left {} ; shape adj {}".format(idx, left_matrix.shape, adj_mat.shape))
                    if verbose:
                        print("\t\tType of left_matrix before = {}".format(type(left_matrix)))
                    left_matrix = left_matrix @ adj_mat
            elif idx <= last_appearance:  # i.e. metanodes[0] = repeatednode
                if verbose:
                    print("\t\tloop_matrix {}".format(idx))
                if dwpc_matrix is None:
                    if verbose:
                        print("\t\tloop_matrix {} initialize; shape adj {}".format(idx, adj_mat.shape))
                    dwpc_matrix = adj_mat.copy()
                else:
                    if verbose:
                        print("\t\tloop_matrix {} multiply; shape loop {} ; shape adj {}".format(idx, dwpc_matrix.shape, adj_mat.shape))
                        print("\t\tType of loop_matrix before = {}".format(type(dwpc_matrix)))
                    dwpc_matrix = dwpc_matrix @ adj_mat
                    # if endpoints are same type, subtract diagonal after mult
                    if metaedge.target == repeated_node[0]:
                        if verbose:
                            print("\tsubtracting diag {} ".format(idx))
                        dwpc_matrix -= scipy.sparse.diags( [dwpc_matrix.diagonal().astype(float)], [0] )
            else:
                if verbose:
                    print("\t\ttail_matrix {}".format(idx))
                dwpc_matrix = dwpc_matrix @ adj_mat

    return left_matrix @ dwpc_matrix

In [38]:
%%time
dwpc_matrix = dwpc(graph, metapath, damping_exponent, False)

def compare_dwpc(output_mat, i, j):
    print("\nCOMPARE")
    print("dwpc_matrix shape {}".format(output_mat.shape))
    print("dwpc from i to j, as computed here:     {}".format(output_mat[i,j]))
    print("dwpc from i to j, as computed by hetio: {}".format(hetio_dwpc))
    print("dwpc from i to j, as computed by cypher: {}".format(cypher_dwpc))

def compare_pc(output_mat, i, j):
    print("\nCOMPARE")
    print("pc_matrix shape {}".format(output_mat.shape))
    print("pc from i to j, as computed here:     {}".format(output_mat[i,j]))
    print("pc from i to j, as computed by hetio: {}".format(len(hetio_paths)))
    print("pc from i to j, as computed by cypher: {}".format(cypher_pc))
    
compare_dwpc(dwpc_matrix, i ,j)

Calling DWPC

COMPARE
dwpc_matrix shape (1552, 137)
dwpc from i to j, as computed here:     0.032875908869216215
dwpc from i to j, as computed by hetio: 0.03287590886921622
dwpc from i to j, as computed by cypher: 0.03287590886921623
CPU times: user 3.32 s, sys: 245 ms, total: 3.57 s
Wall time: 3.58 s


In [39]:
%%time
dwpc_matrix = dwpc(graph, metapath, 0.00, False)

compare_pc(dwpc_matrix, i, j)

Calling DWPC

COMPARE
pc_matrix shape (1552, 137)
pc from i to j, as computed here:     142.0
pc from i to j, as computed by hetio: 142
pc from i to j, as computed by cypher: 142
CPU times: user 3.1 s, sys: 266 ms, total: 3.37 s
Wall time: 3.38 s


In [36]:
3.5 / 1552*137

0.30895618556701027